In [12]:
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
from tqdm import tqdm # Used to display training progress bar
import numpy as np
from scipy import misc
import time

In [3]:
import numpy as np

x = np.ones((1,2,3))

print(x)
print(np.transpose(x,(1,0,2)))

[[[ 1.  1.  1.]
  [ 1.  1.  1.]]]
[[[ 1.  1.  1.]]

 [[ 1.  1.  1.]]]


In [1]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    x_shape = x_tensor.get_shape().as_list()
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.0001)
    
    weights = tf.get_variable('weight', shape= [x_shape[1], num_outputs],
                              initializer=tf.uniform_unit_scaling_initializer(factor=1.0), regularizer =regularizer)
    bias = tf.Variable(tf.zeros([num_outputs]))
    out = tf.add(tf.matmul(x_tensor, weights), bias)
    
    return out

# Functions for constructing layers
def dense_layer(inputs):
    f = flatten(inputs)
    out = output(f, 2)
    return out

In [2]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 1
num_steps = 20
hidden_size = 200
max_epoch = 4
max_max_epoch = 13
keep_prob = 1.0
lr_decay = 0.5
batch_size = 20
vocab_size = 10000

In [ ]:
is_training=1

In [ ]:
# Ops for network architecture
with tf.variable_scope("architecture"):
    # Inputs
    with tf.variable_scope("input"):
        input_data  = tf.placeholder(tf.int32, [num_steps, batch_size], name="data")
        labels = tf.placeholder(tf.int64, [batch_size], name="labels") # Target
        learning_rate =  tf.placeholder(tf.float32, shape =[])
        mask = tf.placeholder(tf.float32, [num_steps, batch_size], name="mask")
        mask = tf.expand_dims(mask, -1)
        
    with tf.variable_scope("lstm"):
        #add LSTM cell and dropout nodes
        cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
        if is_training and keep_prob < 1:
            cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)

    initial_state = cell.zero_state(batch_size, tf.float32)
    with tf.device("/gpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, size])
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        #add dropout to input units
        if is_training and keep_prob < 1:
        inputs = tf.nn.dropout(inputs, keep_prob)

    outputs = []
    state = initial_state
    with tf.variable_scope("RNN"):
        for time_step in range(num_steps):
            if time_step > 0:
                tf.get_variable_scope().reuse_variables()
            (cell_output, state) = cell(inputs[time_step, :, :], state)
            outputs.append(tf.expand_dims(cell_output, 0))
            
            # Write code for synthetic gradient
        
        outputs = tf.concat(0, outputs)*mask
        mask_sum = tf.reduce_sum(mask, 0)
        proj = tf.reduce_sum(outputs, 0)/mask_sum
        #NOW proj has shape [batch_size, size]

        softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        logits = tf.matmul(proj, softmax_w) + softmax_b
        pred = tf.nn.softmax(logits)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels) 
        self.cost = cost = tf.reduce_sum(loss) / batch_size
        self.final_state = state
        correct_prediction = tf.equal(tf.argmax(pred,1), labels)
        self.accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

        if not is_training:
            return


        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),config.max_grad_norm)
        #optimizer = tf.train.GradientDescentOptimizer(self.lr)
        optimizer = tf.train.AdagradOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

In [ ]:
# Ops for network architecture
with tf.variable_scope("architecture"):
    # Inputs
    with tf.variable_scope("input"):
        #X = tf.placeholder(tf.float32, shape=(None, 784), name="data") # Input
        X = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name="data")
        Y = tf.placeholder(tf.float32, shape=(None, 10), name="labels") # Target
        learning_rate =  tf.placeholder(tf.float32, shape =[])
    
    # Inference layers
    with tf.variable_scope("res_block_1"):
        h2 = conv_layer(X, 64, (3,3), (1,1), name="layer1")
        h2_1 = conv_layer(h2, 64, (3,3), (1,1),name="layer2")

    with tf.variable_scope("res_block_2"):
        h3 = conv_layer(h2_1, 128, (3,3), (2,2), name="layer3")
        h3 = conv_layer(h3, 128, (3,3), (1,1), name="layer4")
        h3_1 = conv_layer(h3, 128, (3,3), (1,1), name="layer5")

    with tf.variable_scope("res_block_3"):
        h4 = conv_layer(h3_1, 256, (3,3), (2,2), name="layer6")
        h4 = conv_layer(h4, 256, (3,3), (1,1), name="layer7")
        h4_1 = conv_layer(h4, 256, (3,3), (1,1), name="layer8")

    with tf.variable_scope("res_block_4"):
        h5 = conv_layer(h4_1, 512, (3,3), (2,2), name="layer9")
        h5 = conv_layer(h5, 512, (3,3), (1,1), name="layer10")
        h5 = conv_layer(h5, 512, (3,3), (1,1), name="layer11")
        logits = dense_layer(h5, name="layer12")

    # Synthetic Gradient layers
    with tf.variable_scope("sg_block_1"):
        h_sg1= conv_layer_sg(h2_1, 128, (3,3), (1,1), name="sg1")
        h_sg1 = dense_layer(h_sg1, name="sg2")

    with tf.variable_scope("sg_block_2"):
        h_sg2= conv_layer_sg(h3_1, 128, (3,3), (1,1), name="sg1")
        h_sg2 = dense_layer(h_sg2, name="sg2")
    
    with tf.variable_scope("sg_block_3"):
        h_sg3= conv_layer_sg(h4_1, 128, (3,3), (1,1), name="sg1")
        h_sg3 = dense_layer(h_sg3, name="sg2")
    

# Collections of trainable variables in each block
layer_vars = [tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/res_block_1"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/res_block_2"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/res_block_3"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/res_block_4")]

sg_vars = [tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_1/"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_2/"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_3/")]


In [10]:
batch_size = 128
test_size = 256
# configuration variables
input_vec_size = lstm_size = 28
time_step_size = 28
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, W, B, lstm_size):
    # X, input shape: (batch_size, time_step_size, input_vec_size)
    XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
    # XT shape: (time_step_size, batch_size, input_vec_size)
    XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size=input_vec_size)
    # XR shape: (time_step_size * batch_size, input_vec_size)
    X_split = tf.split(XR, time_step_size, 0) # split them to time_step_size (28 arrays)
    # Each array shape: (batch_size, input_vec_size)

    # Make lstm with lstm_size (each input vector size)
    lstm = rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=True)

    # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = rnn.static_rnn(lstm, X_split, dtype=tf.float32)

    # Linear activation
    # Get the last output
    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat

In [7]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28)
teX = teX.reshape(-1, 28, 28)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
X = tf.placeholder("float", [None, 28, 28])
Y = tf.placeholder("float", [None, 10])

# get lstm_size and output 10 labels
W = init_weights([lstm_size, 10])
B = init_weights([10])

py_x, state_size = model(X, W, B, lstm_size)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [14]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

        test_indices = np.arange(len(teX))  # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices]})))
        print(W.eval())

0 0.51953125
[[  1.60858139e-01  -2.14417711e-01   4.66042198e-02  -4.03741077e-02
   -1.27494648e-01   1.49912313e-01   1.52384102e-01  -2.39743799e-01
    1.62581190e-01  -2.21329793e-01]
 [ -1.47872061e-01   2.86944538e-01  -1.25002608e-01  -5.58807477e-02
   -5.36997393e-02  -1.60194963e-01  -1.54428422e-01   2.66774267e-01
   -1.37787133e-01   6.62228242e-02]
 [ -2.43851691e-01   2.74143547e-01   1.26520604e-01   4.85805534e-02
   -1.44291312e-01  -2.11348534e-01  -1.13123208e-01   5.65850921e-02
   -1.25209078e-01  -1.16104349e-01]
 [  2.51362145e-01   2.00183734e-01  -1.43049389e-01  -1.66712984e-01
   -1.62394285e-01   1.06151514e-02   2.03637451e-01   5.02857454e-02
   -1.41829140e-02  -1.87112898e-01]
 [ -1.87909469e-01   2.68675923e-01   2.69304160e-02   9.88766402e-02
   -3.16060394e-01  -1.94063157e-01   2.67813094e-02   1.74991384e-01
    7.02547841e-05  -2.21089751e-01]
 [ -2.67049849e-01  -2.06651419e-01   1.18466936e-01   2.10385069e-01
    1.43514335e-01  -1.06935389e